In [1]:
import numpy as np
np.__version__

'1.19.2'

# Chapter 11. 넘파이 특화 함수 알아보기  
> 넘파이 모듈에는 벡터화 연산을 위한 유니버셜 함수가 만들어져 있습니다.  이 함수에 대해 먼저 알아보고 파일을 읽고 사용하는 처리도 알아봅시다.

## [복습] 넘파이 모듈의 함수 특징

## 특징 1. 동일한 이름의 함수와 메소드 지원

In [8]:
a = np.zeros((5,5))
(a != 0).any()

False

In [9]:
np.any(a != 0)

False

In [10]:
(a != 0).any

<function ndarray.any>

In [11]:
np.any

<function numpy.any(a, axis=None, out=None, keepdims=<no value>)>

## 특징 2. 유니버셜 함수 제공  
유니버셜(범용) 함수? ndarray 안에 있는 데이터 원소별로 연산을 수행하는 함수.(브로드캐스팅을 지원하는 함수)  
<img src="https://t1.daumcdn.net/cfile/tistory/2330003558BEB65F20" align="left">

In [2]:
np.ufunc

numpy.ufunc

In [14]:
type(np.add) #유니버설 함수

numpy.ufunc

In [16]:
type(np.sort) #일반함수

function

In [19]:
a = np.array([1,2,3,4]) 
np.add(a, a)

array([2, 4, 6, 8])

In [20]:
np.add.accumulate(a) # 일반함수를 사용하면 순환문을 돌려야 함.

array([ 1,  3,  6, 10])

## 특징 3. 벡터화 연산  
브로드캐스팅 : 넘파이에서 shape가 다른 배열간에도 산술 연산이 가능하게 하는 매커니즘. 

In [24]:
x = np.array([[1,2,3], [4,5,6]])
y = np.array([1,0,1])
x + y # 차원이 다른 y를 np.tile(y, (4, 1))로 처리 (y의 복사본 4개를 쌓음)

array([[2, 2, 4],
       [5, 5, 7]])

In [29]:
x = np.array([[1,2,3], [4,5,6]])
y = np.array([1,0])
x + y

ValueError: operands could not be broadcast together with shapes (2,3) (2,) 

## 01. 일반함수와 람다함수 사용하기.  
ufunc 클래스로 만든 함수를 처리하는 방식을 알아봤습니다.  
이제 이 클래스에서 제공하는 메소드를 알아본 후 함수를 벡터화 연산 처리 하는 방식을 알아봅시다.

### 예제 1. 유니버설 함수 클래스 알아보기
[유니버셜 함수 레퍼런스](https://docs.scipy.org/doc/numpy/reference/ufuncs.html)

In [37]:
type(np.add) # 유니버설 함수의 클래스다.

numpy.ufunc

In [36]:
# 유니버셜 함수의 메소드
for i in dir(np.ufunc) :
    if not i.startswith("_") :
        print(i)

accumulate
at
identity
nargs
nin
nout
ntypes
outer
reduce
reduceat
signature
types


### 예제 2. add 함수를 사용해서 메소드 처리하기

In [43]:
a = np.arange(10)
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [44]:
import functools as ft # 고차함수를 위한 라이브러리(기존 함수를 모아놓은 모듈)

# reduce(집계함수, 순회 가능한 데이터, [초기값-생략가능])
# 특정함수와 반복형 객체를 받아 연산을 처리하고 결과를 반환해 줌

ft.reduce(np.add, a) 

45

In [45]:
np.add.reduce(a)

45

In [46]:
a.sum()

45

In [47]:
# reduceat 메소드로 배열의 특정 위치를 지정해서 합산
np.add.reduceat(a, [0, 4])

array([ 6, 39])

In [48]:
np.add.reduce(a[:4])

6

In [49]:
np.add.reduce(a[4:])

39

In [50]:
np.add.reduceat(a, [0, 4, 1, 5])

array([ 6,  4, 10, 35])

In [51]:
print(np.add.reduce(a[:4]))
print(np.add.reduce(a[4:5])) # 인덱스 주의!
print(np.add.reduce(a[1:5]))
print(np.add.reduce(a[5:]))

6
4
10
35


## Quiz 1
np.add.reduceat(np.arange(8), [0, 4, 1, 5, 2, 6, 3, 7])[::2]  
array([6, 10, 14, 18])  
위 결과값을 나온 이유를 설명해 주세요.

In [44]:
print(np.add.reduceat([0,1,2,3,4,5,6,7], [0, 4]))
print(np.add.reduceat([0,1,2,3,4,5,6,7], [4, 1]))
print(np.add.reduceat([0,1,2,3,4,5,6,7], [1, 5]))
print(np.add.reduceat([0,1,2,3,4,5,6,7], [5, 2]))
print(np.add.reduceat([0,1,2,3,4,5,6,7], [2, 6]))
print(np.add.reduceat([0,1,2,3,4,5,6,7], [6, 3]))
print(np.add.reduceat([0,1,2,3,4,5,6,7], [3, 7]))

x = np.add.reduceat([0,1,2,3,4,5,6,7], [0, 4, 1, 5, 2, 6, 3, 7])
print(x)

x[::2]

[ 6 22]
[ 4 28]
[10 18]
[ 5 27]
[14 13]
[ 6 25]
[18  7]
[ 6  4 10  5 14  6 18  7]


array([ 6, 10, 14, 18])

### 예제3. logical_and 함수로 알아보기

In [45]:
type(np.logical_and)

numpy.ufunc

In [49]:
y = np.array([[2, 5], [7, 8]])
z = np.array([[2, 3], [2, 8]])
a = y > z
a

array([[False,  True],
       [ True, False]])

In [47]:
b = np.logical_and.reduce(y>z) # 행에 있는 논리값들을 다시 비교해서 결과 출력
b

array([False, False])

In [51]:
# 논리값의 결과를 더 줄이는 .any와 .all
(y>z).all()

False

In [52]:
(y>z).any()

True

### 사용자 유니버셜 함수 정의 하기

In [53]:
np.frompyfunc # 함수를 등록하는 frompyfunc 함수

<function numpy.frompyfunc>

In [76]:
import operator as op

def add(x, y) :
    return op.add(x, y)

type(add)

function

In [96]:
a = np.arange(3)
b = np.arange(4, 7)
c = add(a, b)
c

array([4, 6, 8])

In [97]:
add_ = np.frompyfunc(add, 2, 1)  #frompyfunc(함수, 입력 인자 개수, 출력 객체수)
type(add_)

numpy.ufunc

In [98]:
d = add_(a, b)
d

array([4, 6, 8], dtype=object)

In [80]:
%timeit add_(a, b) # 오브젝트 타입으로 처리되어 느림

1.67 µs ± 19.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [81]:
%timeit add(a, b) # 스페셜 메소드로 처리하는것이 빠름

462 ns ± 4.56 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [103]:
np.vectorize #벡터화 함수를 만드는 vectorize 클래스

numpy.vectorize

In [99]:
def mul(x, y) :
    return op.mul(x, y)

type(mul)

function

In [100]:
e = mul(a, b)
e

array([ 0,  5, 12])

In [106]:
mul_ = np.vectorize(mul) # 참조 https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html
type(mul_)

numpy.vectorize

In [107]:
f = mul_(a, b)
f

array([ 0,  5, 12])

In [108]:
%timeit mul_(a, b) # 벡터화 오브젝트 타입으로 처리되어 느림

10.5 µs ± 261 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [109]:
%timeit mul(a, b) # 스페셜 메소드로 처리하는것이 빠름

455 ns ± 6.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### 함수를 전달 받아 배열 생성하기

In [111]:
np.fromfunction

<function numpy.fromfunction(function, shape, *, dtype=<class 'float'>, **kwargs)>

In [112]:
type(np.fromfunction)

function

In [113]:
a = np.fromfunction(lambda i, j : i == j, (3, 3)) # 람다 매개변수는 배열의 좌표입니다.
a

array([[ True, False, False],
       [False,  True, False],
       [False, False,  True]])

In [120]:
b = np.fromfunction(lambda i, j : i+j, (3, 3))
b

array([[0., 1., 2.],
       [1., 2., 3.],
       [2., 3., 4.]])

## 02. 파일처리

### 02-1. 일반파일 처리하기

In [121]:
n = np.random.randn(3, 5)
n

array([[-1.03920641,  0.25416194,  0.48729407, -0.50420902, -3.58750699],
       [ 2.44926666,  1.24281422, -0.902584  ,  0.76763889, -1.8285857 ],
       [-1.10399085,  0.71767767,  1.82920702, -1.09855516,  0.57905975]])

In [122]:
n.tofile('n.npy')

In [125]:
%ls n.*

n.npy


In [127]:
m = np.fromfile('n.npy')
m

array([-1.03920641,  0.25416194,  0.48729407, -0.50420902, -3.58750699,
        2.44926666,  1.24281422, -0.902584  ,  0.76763889, -1.8285857 ,
       -1.10399085,  0.71767767,  1.82920702, -1.09855516,  0.57905975])

In [128]:
np.save('m.npy', m) # 함수로 처리
%ls m.*

m.npy


In [129]:
o = np.load('m.npy')
o

array([-1.03920641,  0.25416194,  0.48729407, -0.50420902, -3.58750699,
        2.44926666,  1.24281422, -0.902584  ,  0.76763889, -1.8285857 ,
       -1.10399085,  0.71767767,  1.82920702, -1.09855516,  0.57905975])

### 02-2. 텍스트 파일 처리하기

In [130]:
a = np.random.randn(3, 3)
a

array([[ 0.0232927 , -1.01570736, -0.33861738],
       [ 0.88726319, -0.61807078, -0.03592853],
       [ 1.18483685, -1.3631187 , -1.28021693]])

In [132]:
np.savetxt('data.txt', a)
%ls data.*

data.txt


In [ ]:
# %load data.txt
2.329270165218954095e-02 -1.015707363147797748e+00 -3.386173777006352692e-01
8.872631866260904943e-01 -6.180707826644971092e-01 -3.592853432097895677e-02
1.184836852830924547e+00 -1.363118698625313208e+00 -1.280216926978795522e+00


In [137]:
b = np.loadtxt('data.txt')
b

array([[ 0.0232927 , -1.01570736, -0.33861738],
       [ 0.88726319, -0.61807078, -0.03592853],
       [ 1.18483685, -1.3631187 , -1.28021693]])

In [138]:
c = a + b
np.savetxt('data.txt', c)
d = np.loadtxt('data.txt')
d

array([[ 0.0465854 , -2.03141473, -0.67723476],
       [ 1.77452637, -1.23614157, -0.07185707],
       [ 2.36967371, -2.7262374 , -2.56043385]])

### 02-3. csv 파일 처리하기

In [140]:
%%writefile file.dat
1 2 3
4 5 6
7 8 9

Overwriting file.dat


In [141]:
data = np.genfromtxt('file.dat')
data

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

In [142]:
np.savetxt('file.csv', data)

In [ ]:
# %load file.csv
1.000000000000000000e+00 2.000000000000000000e+00 3.000000000000000000e+00
4.000000000000000000e+00 5.000000000000000000e+00 6.000000000000000000e+00
7.000000000000000000e+00 8.000000000000000000e+00 9.000000000000000000e+00


In [145]:
a = np.loadtxt('file.csv')
a

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

### 02-4 직렬화 처리  
npz 확장자를 사용하여 다차원 배열을 직렬화 처리하고 저장

In [146]:
x = np.random.randn(3, 5)
x

array([[-2.2680059 ,  0.52827395,  0.69363968,  1.92301472, -1.7353325 ],
       [-1.34918042, -1.19615115, -0.7221849 , -0.88853132, -0.96246549],
       [-0.52816851,  1.12375436,  1.53145049,  0.4935921 , -0.26263873]])

In [147]:
np.savez('xxx', x) # 파일이름과 배열을 인자로 전달, npz 확장자를 지정하지 않아도 npz파일이 만들어짐.
%ls xxx.*

xxx.npz


In [148]:
y = np.load('xxx.npz')

In [149]:
type(y)

numpy.lib.npyio.NpzFile

In [150]:
y.files

['arr_0']

In [151]:
y['arr_0']

array([[-2.2680059 ,  0.52827395,  0.69363968,  1.92301472, -1.7353325 ],
       [-1.34918042, -1.19615115, -0.7221849 , -0.88853132, -0.96246549],
       [-0.52816851,  1.12375436,  1.53145049,  0.4935921 , -0.26263873]])

In [152]:
# 다양한 속성과 메소드를 제공
for i in dir(y) :
    if not i.startswith("_") :
        print(i)

allow_pickle
close
f
fid
files
get
items
iteritems
iterkeys
keys
pickle_kwargs
values
zip


In [153]:
# savez를 쓰는 이유 : Save several arrays into a single file in uncompressed .npz format.
z = np.random.randn(3, 3)
z

array([[-0.34013116, -1.34136584, -2.3307599 ],
       [-1.32950889, -0.0708689 , -2.5587767 ],
       [ 0.43589296, -0.26617065,  0.47097827]])

In [166]:
np.savez('xxx', x, z)

In [167]:
z = np.load('xxx.npz')

In [168]:
z.files

['arr_0', 'arr_1']